In [ ]:
import pandas as pd
import numpy as np

# Генерация фальшивых данных
def generate_fake_data(num_samples=1000):
    np.random.seed(42)
    y_true = np.random.randint(0, 2, size=num_samples)  # Истинные значения (0 или 1)
    y_pred = np.random.randint(0, 2, size=num_samples)  # Предсказанные значения (0 или 1)
    
    # Создание DataFrame
    data = pd.DataFrame({
        'true_label': y_true,
        'predicted_label': y_pred
    })
    
    return data

fake_data = generate_fake_data()

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, f1_score

data = fake_data

# Расчет accuracy и recall
accuracy = accuracy_score(data['true_label'], data['predicted_label'])
recall = recall_score(data['true_label'], data['predicted_label'])
f1 = f1_score(data['true_label'], data['predicted_label'])

print(f'Accuracy: {accuracy:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1: {f1:.2f}')

Accuracy: 0.50
Recall: 0.48
F1: 0.49
